# Interpreting Stoplight Meanings

In [1]:
import numpy as np

In [2]:
streetlight_states = np.array(
    [
        [1, 0, 1],
        [0, 1, 1],
        [0, 0, 1],
        [1, 1, 1],
        [0, 1, 1],
        [1, 0, 1],
    ]
)

walk_states = np.array(
    [
        [0],
        [1],
        [0],
        [1],
        [1],
        [0],
    ]
)

In [3]:
weights = np.array([0.5, 0.2, 0.3])
alpha = 0.1

input = streetlight_states[0]
goal_prediction=walk_states[0]

for i in range(20):
    prediction = input.dot(weights)
    error = (goal_prediction - prediction) ** 2
    delta = prediction - goal_prediction
    weights = weights - (alpha * (input * delta))
    print(f"Error: {error} Prediction: {prediction}")

Error: [0.64] Prediction: 0.8
Error: [0.4096] Prediction: 0.6399999999999999
Error: [0.262144] Prediction: 0.512
Error: [0.16777216] Prediction: 0.40959999999999996
Error: [0.10737418] Prediction: 0.32767999999999997
Error: [0.06871948] Prediction: 0.26214399999999993
Error: [0.04398047] Prediction: 0.20971519999999996
Error: [0.0281475] Prediction: 0.16777215999999998
Error: [0.0180144] Prediction: 0.13421772799999995
Error: [0.01152922] Prediction: 0.10737418239999996
Error: [0.0073787] Prediction: 0.08589934591999997
Error: [0.00472237] Prediction: 0.06871947673599997
Error: [0.00302231] Prediction: 0.054975581388799974
Error: [0.00193428] Prediction: 0.04398046511103998
Error: [0.00123794] Prediction: 0.03518437208883199
Error: [0.00079228] Prediction: 0.028147497671065586
Error: [0.00050706] Prediction: 0.022517998136852457
Error: [0.00032452] Prediction: 0.01801439850948197
Error: [0.00020769] Prediction: 0.014411518807585566
Error: [0.00013292] Prediction: 0.011529215046068464


But how do we train a model on the full set?

In [4]:
weights = np.random.rand(3)
alpha = 0.1

input = streetlight_states[0]
goal_prediction=walk_states[0]

for i in range(40):
    error_for_all_lights = 0
    for row_index in range(len(streetlight_states)):
        input = streetlight_states[row_index]
        goal_prediction = walk_states[row_index]

        prediction = input.dot(weights)

        error = (goal_prediction - prediction) ** 2
        error_for_all_lights += error

        delta = prediction - goal_prediction
        weights = weights - (alpha * (input * delta))
        print(f"Prediction: {prediction}")
    print(f"Error: {error} Prediction: {error_for_all_lights}")

Prediction: 1.2548748430623422
Prediction: 1.2946840400269766
Prediction: 0.6857257019864702
Prediction: 1.4558804302836403
Prediction: 1.075998575766206
Prediction: 0.8070829566151804
Error: [0.6513829] Prediction: [2.99675494]
Prediction: 0.6456663652921443
Prediction: 0.9155239284222324
Prediction: 0.42713790614988273
Prediction: 0.987548145364407
Prediction: 0.8921957230499162
Prediction: 0.49553770739863084
Error: [0.24555762] Prediction: [0.86380248]
Prediction: 0.39643016591890473
Prediction: 0.8245597911081795
Prediction: 0.32479696225089066
Prediction: 0.8370593280348697
Prediction: 0.8597562710544806
Prediction: 0.3488209646867948
Error: [0.12167607] Prediction: [0.46132324]
Prediction: 0.2790567717494359
Prediction: 0.8250172431999615
Prediction: 0.2773462081532474
Prediction: 0.7956766586709496
Prediction: 0.8731438420104545
Prediction: 0.2665593563289924
Error: [0.07105389] Prediction: [0.31430697]
Prediction: 0.2132474850631939
Prediction: 0.850534389469145
Prediction: 0.

In [5]:
weights

array([-0.01716005,  0.98264083,  0.01992276])

# Let's make a deep neural net now

In [6]:
def relu(x):
    return (x > 0) * x

alpha = 0.2
hidden_size = 4

weights_0_1 = 2 * np.random.random((3, hidden_size)) - 1
weights_1_2 = 2 * np.random.random((hidden_size, 1)) - 1

layer_0 = streetlight_states[0]
layer_1 = relu(layer_0.dot(weights_0_1))
layer_2 = np.dot(layer_1, weights_1_2)

In [7]:
layer_0

array([1, 0, 1])

In [8]:
layer_1

array([-0.       , -0.       ,  0.0118443, -0.       ])

In [9]:
layer_2


array([0.00518332])

In [10]:
def relu2deriv(output):
    return output > 0

In [16]:
weights_0_1 = 2 * np.random.random((3, hidden_size)) - 1
weights_1_2 = 2 * np.random.random((hidden_size, 1)) - 1

for iteration in range(60):
    layer_2_error = 0
    for i in range(len(streetlight_states)):
        layer_0 = streetlight_states[i:i+1]
        layer_1 = relu(np.dot(layer_0, weights_0_1))
        layer_2 = np.dot(layer_1, weights_1_2)

        layer_2_error += np.sum((layer_2 - walk_states[i:i+1]) ** 2)

        layer_2_delta = (layer_2 - walk_states[i:i+1])
        layer_1_delta = layer_2_delta.dot(weights_1_2.T) * relu2deriv(layer_1)

        weights_1_2 -= alpha * layer_1.T.dot(layer_2_delta)
        weights_0_1 -= alpha * layer_0.T.dot(layer_1_delta)
    
    if iteration % 10 == 9:
        print(f"Error: {layer_2_error}")

Error: 0.11384873484291419
Error: 0.010628167601735702
Error: 0.0006400857489958602
Error: 3.281269605975691e-05
Error: 1.6145505410640569e-06
Error: 7.870510196276315e-08
